In [25]:
# Célula 1: Importar as bibliotecas necessárias
import pandas as pd
import numpy as np
import os
import openpyxl
import xlsxwriter
import glob
import hashlib # Importa a biblioteca para fazer o hash


In [ ]:
dicionario = {
    'Unnamed: 1_level_1': 'total',
    'Unnamed: 1': 'total',
    'total(1)': 'total',
    'sexo': 'homens',
    'sexo.1': 'mulheres',
    'cor ou raça': 'branca',
    'cor ou raça.1': 'preta/parda',
    'Preta ou parda ': 'preta/parda',
    'Preta ou parda .1': 'preta',
    'Preta ou parda .2': 'parda',
    'cor ou raça.2': 'preta',
    'cor ou raça.3': 'parda',
    'Sem rendimento a menos de 1/4 (2) ': 'sem rendimento a menos de 1/4',
    'sem rendimento a menos de 1/4 (2)': 'sem rendimento a menos de 1/4',
    'Unnamed: 5_level_1': 'total',
    'realização do registro da última agressão física na delegacia de polícia': 'realizaram',
    'realização do registro da última agressão física na delegacia de polícia.1': 'nao realizaram',
    'local de ocorrência da última agressão física': 'propria residencia',
    'local de ocorrência da última agressão física.1': 'via publica',
    'local de ocorrência da última agressão física.2': 'outro',
    'motivo de não terem procurado a polícia em decorrência da última agressão física': 'falta de provas',
    'motivo de não terem procurado a polícia em decorrência da última agressão física.1': 'nao era importante',
    'motivo de não terem procurado a polícia em decorrência da última agressão física.2': 'nao acreditavam na policia',
    'motivo de não terem procurado a polícia em decorrência da última agressão física.3': 'nao queriam envolver a policia ou medo de represalias',
    'motivo de não terem procurado a polícia em decorrência da última agressão física.4': 'outro',
    'grupos de idade': '10 a 15 anos',
    'grupos de idade.1': '16 a 24 anos',
    'grupos de idade.2': '25 a 34 anos',
    'grupos de idade.3': '35 a 49 anos',
    'grupos de idade.4': '50 a 59 anos(ou mais)',
    'grupos de idade.5': '60 a 69 anos',
    'grupos de idade.6': '70 anos ou mais',
    'economicamente ativas': 'economicamente ativas',
    'economicamente ativas.1': 'condição de ocupação na semana de referência(ocupadas)',
    'economicamente ativas.2': 'condição de ocupação na semana de referência(desocupadascupadas)',
    'fundamental ou equivalente':'fundamental incompleto ou equivalente',
    'fundamental ou equivalente.1':'fundamental completo ou equivalente',
    'médio ou equivalente':'médio incompleto ou equivalente',
    'médio ou equivalente.1':'médio completo ou equivalente',
    'superior ou equivalente':'superior incompleto ou equivalente',
    'superior ou equivalente.1':'superior completo ou equivalente',
}

In [27]:
# Função: Carregamento de dados CSV/XLSX
caminho_pasta = '../../dados_brutos/PNAD_2009'

agressao = glob.glob(os.path.join(caminho_pasta, 'agressao/*.xls'), recursive=True)
furto = glob.glob(os.path.join(caminho_pasta, 'furto/*.xls'), recursive=True)
roubo = glob.glob(os.path.join(caminho_pasta, 'roubo/*.xls'), recursive=True)
roubofurto = glob.glob(os.path.join(caminho_pasta, 'roubofurto/*.xls'), recursive=True)
seguranca = glob.glob(os.path.join(caminho_pasta, 'seguranca/*.xls'), recursive=True)
tentativa = glob.glob(os.path.join(caminho_pasta, 'tentativa/*.xls'), recursive=True)

def processar_arquivo(tipo_arquivos, categoria):
    dados = {}

    for arquivo in tipo_arquivos:
        try:
            # --- Leitura do nome_base (lógica mantida) ---
            df_temp = pd.read_excel(arquivo, sheet_name=0, header=None, nrows=5)
            valor_celula = str(df_temp.iloc[2, 0]).strip().split("-", 1)[1] + " " + str(df_temp.iloc[3, 0]).strip() + " " + str(df_temp.iloc[4, 0]).strip()
            
            if pd.isna(valor_celula) or valor_celula == "" or valor_celula.lower() == "nan":
                nome_base = os.path.basename(arquivo).replace('.xlsx', '').replace('.xls', '')
                print(f"[AVISO] Célula de nome vazia para {arquivo}. Usando nome do arquivo como fallback: {nome_base}")
            else:
                nome_base = valor_celula.split("-", 1)[0].strip()

            xls = pd.ExcelFile(arquivo)
            for sheet_name in xls.sheet_names:
                df_nome = f"{nome_base}_{sheet_name}" if len(xls.sheet_names) > 1 else nome_base

                # --- NOVA LÓGICA DE DETECÇÃO ---
                # ETAPA 1: "Espiar" a primeira linha do cabeçalho potencial (linha 7 do Excel) para verificar a presença de "unnamed:".
                # Usamos header=None para ler a linha como dados brutos.
                df_peek = pd.read_excel(arquivo, sheet_name=sheet_name, header=0, index_col=0, skiprows=7, nrows=1)
                df_peek.rename(columns={
                    'Unnamed: 1_level_1': 'total',
                    'Unnamed: 1': 'total',
                    'Unnamed: 5': 'total',
                }, inplace=True)
                print(df_peek)
                # Pega os nomes das colunas que o pandas leu e os converte para minúsculas
                peek_columns = df_peek.columns.str.lower().tolist()
                print(peek_columns)

                # ETAPA 2: Decidir a estratégia com base na presença de "unnamed:"
                # Verifica se a coluna começa ou termina com o texto "unnamed:"
                needs_multi_index = any("unnamed:" in col for col in peek_columns)

                # ETAPA 3: Carregar o DataFrame UMA ÚNICA VEZ com os parâmetros corretos.
                if needs_multi_index:
                    # ETAPA 3A: Lógica para Multi-Index
                    print(f"-> Detectado 'Unnamed:'. Aplicando MultiIndex [0,1] e skiprows=6 para {sheet_name}")
                    df = pd.read_excel(arquivo, sheet_name=sheet_name, header=[0, 1], index_col=0, skipfooter=3, skiprows=6)
                else:
                    # ETAPA 3B: Lógica para Index Simples
                    print(f"-> Header OK. Aplicando Index simples e skiprows=7 para {sheet_name}")
                    df = pd.read_excel(arquivo, sheet_name=sheet_name, header=0, index_col=0, skipfooter=3, skiprows=7)

                # --- Tratamento dos dados ---
                # Este dicionário de renomeação será aplicado em ambos os casos.
                # Ele renomeará as colunas que encontrar, seja em um Index ou MultiIndex.
                df.rename(columns=dicionario, inplace=True)

                df = df.dropna(how='all')

                # --- Limpeza de Colunas Específica para cada caso ---
                if isinstance(df.columns, pd.MultiIndex):
                    # Limpa cada nível do MultiIndex
                    df.columns = pd.MultiIndex.from_tuples([
                        tuple(str(level).strip().lower().replace('\n', '') for level in col)
                        for col in df.columns
                    ])
                    df.columns = df.columns.get_level_values(1)
                else:
                    # Limpa o Index simples
                    df.columns = df.columns.astype(str).str.replace('\n', '', regex=True).str.strip().str.lower()

                df.rename(columns=dicionario, inplace=True)

                # --- Tratamento Comum ---
                df = df[df.index.notna()]
                df.index = df.index.astype(str).str.lower()

                for col_name in df.columns:
                    series = df[col_name]
                    if pd.api.types.is_string_dtype(series):
                        df[col_name] = series.str.lower()

                df = df.apply(pd.to_numeric, errors='coerce')
                df = df.round(2)

                dados[df_nome] = df

                print(f"[OK] Excel {df_nome}")

        except Exception as e:
            print(f"[ERRO] Excel {arquivo}: {e}")

    return dados

dados_agressao = processar_arquivo(agressao, 'agressao')
dados_furto = processar_arquivo(furto, 'furto')
dados_roubo = processar_arquivo(roubo, 'roubo')
dados_roubofurto = processar_arquivo(roubofurto, 'roubofurto')
dados_seguranca = processar_arquivo(seguranca, 'seguranca')
dados_tentativa = processar_arquivo(tentativa, 'tentativa')

     Total\n(1)    Sexo Unnamed: 3 Cor ou raça            total
NaN         NaN  Homens   Mulheres      Branca  Preta ou parda 
['total\n(1)', 'sexo', 'unnamed: 3', 'cor ou raça', 'total']
-> Detectado 'Unnamed:'. Aplicando MultiIndex [0,1] e skiprows=6 para Tab 1.2.6.1.1
[OK] Excel Coeficientes de variação das estimativas de pessoas de 10 anos ou mais de idade que foram vítimas de agressão física, no período de referência de 365 dias, por sexo e cor ou raça, segundo a situação do domicílio, as Grandes Regiões e as Unidades da Federação
     Total\n(1)    Sexo Unnamed: 3 Cor ou raça            total
NaN         NaN  Homens   Mulheres      Branca  Preta ou parda 
['total\n(1)', 'sexo', 'unnamed: 3', 'cor ou raça', 'total']
-> Detectado 'Unnamed:'. Aplicando MultiIndex [0,1] e skiprows=6 para Tab 1.2.6.1.2
[OK] Excel Coeficientes de variação das estimativas de distribuição das pessoas de 10 anos ou mais de idade que foram vítimas de agressão física, no período de referência de 365 dias, 

In [28]:
display(dados_agressao['Coeficiente de variação das estimativas de pessoas de 10 anos ou mais de idade que foram vítimas de agressão física, no período de referência de 365 dias, por sexo e cor ou raça, segundo algumas Unidades da Federação e Regiões Metropolitanas'])

,total,homens,mulheres,branca,preta/parda
pará,7.58,7.90,11.18,15.12,7.49
região metropolitana de belém,8.57,10.07,11.52,16.38,9.57
ceará,8.67,8.19,13.53,14.51,8.98
região metropolitana de fortaleza,7.83,9.27,11.40,13.59,8.95
pernambuco,6.58,7.96,9.18,11.82,7.91
região metropolitana de recife,8.64,11.01,12.52,12.90,11.22
bahia,6.10,7.01,8.32,15.22,5.97
região metropolitana de salvador,6.23,7.76,9.18,13.71,6.63
minas gerais,7.32,8.49,8.57,11.86,7.64
região metropolitana de belo horizonte,9.10,11.38,12.17,17.64,10.36


In [29]:
display(dados_roubofurto['Coeficientes de variação das estimativas de distribuição das pessoas de 10 anos ou mais de idade que foram vítimas de roubo ou furto, no período de referência de 365 dias, por sexo e cor ou raça, segundo algumas Unidades da Federação e Regiões Metropolitanas'].info())
display(dados_roubofurto['Coeficientes de variação das estimativas de distribuição das pessoas de 10 anos ou mais de idade que foram vítimas de roubo ou furto, no período de referência de 365 dias, por sexo e cor ou raça, segundo algumas Unidades da Federação e Regiões Metropolitanas'].head())

<class 'pandas.core.frame.DataFrame'>
Index: 18 entries, pará to         região metropolitana de porto alegre
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   total        18 non-null     float64
 1   homens       18 non-null     float64
 2   mulheres     18 non-null     float64
 3   branca       18 non-null     float64
 4   preta/parda  18 non-null     float64
 5   preta        18 non-null     float64
 6   parda        18 non-null     float64
dtypes: float64(7)
memory usage: 1.7+ KB


None

,total,homens,mulheres,branca,preta/parda,preta,parda
pará,0.0,2.21,2.58,5.73,1.84,9.94,2.18
região metropolitana de belém,0.0,2.50,2.87,4.81,1.67,9.73,1.95
ceará,0.0,2.01,2.50,4.29,1.96,17.30,2.31
região metropolitana de fortaleza,0.0,2.14,2.50,3.92,2.02,12.84,2.23
pernambuco,0.0,2.46,2.89,3.50,2.26,10.21,2.56


In [30]:
display(dados_agressao['Coeficientes de variação das estimativas de distribuição das pessoas de 10 anos ou mais de idade que foram vítimas de agressão física, no período de referência de 365 dias, por agressor na última agressão física, segundo a situação do domicílio e as Grandes Regiões'].info())
display(dados_agressao['Coeficientes de variação das estimativas de distribuição das pessoas de 10 anos ou mais de idade que foram vítimas de agressão física, no período de referência de 365 dias, por agressor na última agressão física, segundo a situação do domicílio e as Grandes Regiões'].head())


<class 'pandas.core.frame.DataFrame'>
Index: 7 entries,                brasil to sul
Data columns (total 6 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   total                                 7 non-null      float64
 1   agressor na última agressão física    7 non-null      float64
 2   agressor na última agressão física.1  7 non-null      float64
 3   agressor na última agressão física.2  7 non-null      float64
 4   agressor na última agressão física.3  7 non-null      float64
 5   agressor na última agressão física.4  7 non-null      float64
dtypes: float64(6)
memory usage: 692.0+ bytes


None

,total,agressor na última agressão física,agressor na última agressão física.1,agressor na última agressão física.2,agressor na última agressão física.3,agressor na última agressão física.4
brasil,0.0,2.24,7.56,4.33,5.69,2.31
urbana,0.0,2.30,7.79,4.58,5.99,2.43
rural,0.0,8.24,26.07,11.86,14.68,6.21
norte,0.0,4.76,19.23,9.77,14.13,5.88
nordeste,0.0,3.94,13.48,7.26,9.59,3.73


In [31]:
todos_dados = {
    'agressao': dados_agressao,
    'furto': dados_furto,
    'roubo': dados_roubo,
    'roubofurto': dados_roubofurto,
    'seguranca': dados_seguranca,
    'tentativa': dados_tentativa
}


In [32]:
def exportar_dados_organizados(dicionario_principal, pasta_base='../../dados_tratados/PNAD_2009'):
    """
    Exporta um dicionário aninhado de DataFrames para arquivos CSV e XLSX,
    organizando os arquivos em pastas por categoria e por formato.
    
    Esta versão usa hashing (MD5) para encurtar nomes de arquivo muito longos
    e cria um arquivo de mapeamento ('_mapa_de_nomes.csv') para manter a 
    referência aos nomes originais.
    """
    print(f"Iniciando a exportação para a pasta base: '{pasta_base}'")
    
    if not dicionario_principal:
        print("[AVISO] O dicionário de dados está vazio. Nenhuma exportação será feita.")
        return

    # Loop pelas categorias (ex: 'agressao', 'furto')
    for categoria, dicionario_dataframes in dicionario_principal.items():
        print(f"\n--- Exportando categoria: {categoria.upper()} ---")
        
        # Define e cria os diretórios de saída
        path_saida_csv = os.path.join(pasta_base, 'csv', categoria)
        path_saida_xlsx = os.path.join(pasta_base, 'xlsx', categoria)
        os.makedirs(path_saida_csv, exist_ok=True)
        os.makedirs(path_saida_xlsx, exist_ok=True)
        
        # NOVO: Lista para guardar o mapeamento dos nomes
        mapeamento_nomes = []
        
        # Loop pelos DataFrames dentro da categoria
        for nome_original_df, df in dicionario_dataframes.items():
            
            # NOVO: Gera um nome de arquivo curto e único usando hash MD5
            # O .encode('utf-8') é importante para a função de hash funcionar corretamente com qualquer caractere
            nome_curto_hash = hashlib.md5(nome_original_df.encode('utf-8')).hexdigest()
            
            # NOVO: Adiciona o mapeamento à nossa lista
            mapeamento_nomes.append({
                'nome_original': nome_original_df,
                'nome_arquivo_hash': nome_curto_hash
            })
            
            # --- Exportação para CSV ---
            # USA O NOME CURTO para salvar o arquivo
            caminho_csv = os.path.join(path_saida_csv, f"{nome_curto_hash}.csv")
            try:
                df.to_csv(caminho_csv, sep=';', encoding='utf-8-sig', index=True)
                print(f"  [OK] CSV salvo: {caminho_csv}")
            except Exception as e:
                print(f"  [ERRO] Falha ao salvar CSV {caminho_csv}: {e}")

            # --- Exportação para XLSX ---
            # USA O NOME CURTO para salvar o arquivo
            caminho_xlsx = os.path.join(path_saida_xlsx, f"{nome_curto_hash}.xlsx")
            try:
                df.to_excel(caminho_xlsx, index=True, sheet_name='dados')
                print(f"  [OK] XLSX salvo: {caminho_xlsx}")
            except Exception as e:
                print(f"  [ERRO] Falha ao salvar XLSX {caminho_xlsx}: {e}")
        
        # NOVO: Ao final de cada categoria, salva o arquivo de mapeamento
        if mapeamento_nomes:
            try:
                df_mapa = pd.DataFrame(mapeamento_nomes)
                # Salva o mapa na pasta CSV da categoria para fácil acesso
                caminho_mapa_csv = os.path.join(path_saida_csv, '_mapa_de_nomes.csv')
                df_mapa.to_csv(caminho_mapa_csv, sep=';', encoding='utf-8-sig', index=False)
                print(f"\n  [INFO] Arquivo de mapeamento de nomes salvo em: {caminho_mapa_csv}")
                
                # Opcional: Salvar também na pasta XLSX
                caminho_mapa_xlsx = os.path.join(path_saida_xlsx, '_mapa_de_nomes.xlsx')
                df_mapa.to_excel(caminho_mapa_xlsx, index=False)

            except Exception as e:
                print(f"  [ERRO] Falha ao salvar o arquivo de mapeamento: {e}")
                
    print(f"\n[SUCESSO] Exportação de todos os dados concluída!")
    print(f"Verifique a pasta '{pasta_base}' no seu diretório.")

In [33]:
import pandas as pd
import os
import hashlib

def exportar_dados_organizados_excel_unico(dicionario_principal, pasta_base='../../dados_tratados/PNAD_2009'):
    """
    Exporta um dicionário aninhado de DataFrames para arquivos Excel (um por categoria),
    com cada DataFrame em uma aba e o índice exportado como a primeira coluna.
    Gera também um mapeamento de nomes originais para as abas do Excel.
    """
    print(f"Iniciando a exportação para a pasta base: '{pasta_base}'")
    
    if not dicionario_principal:
        print("[AVISO] O dicionário de dados está vazio. Nenhuma exportação será feita.")
        return

    for categoria, dicionario_dataframes in dicionario_principal.items():
        print(f"\n--- Exportando categoria: {categoria.upper()} ---")

        path_saida_xlsx = os.path.join(pasta_base, 'xlsx_unico')
        os.makedirs(path_saida_xlsx, exist_ok=True)

        caminho_arquivo_categoria = os.path.join(path_saida_xlsx, f"{categoria}.xlsx")

        writer = pd.ExcelWriter(caminho_arquivo_categoria, engine='xlsxwriter')
        mapeamento_nomes = []

        for nome_original, df in dicionario_dataframes.items():
            # Nome da aba com até 31 caracteres
            nome_limpo = nome_original.strip().replace("/", "-").replace("\\", "-")
            if len(nome_limpo) > 31:
                nome_hash = hashlib.md5(nome_original.encode('utf-8')).hexdigest()[:8]
                nome_aba = nome_limpo[:22] + '_' + nome_hash
            else:
                nome_aba = nome_limpo

            # Adiciona ao mapeamento
            mapeamento_nomes.append({
                'nome_original': nome_original,
                'nome_aba': nome_aba
            })

            # Insere o index como coluna
            df_exportar = df.copy()
            df_exportar.insert(0, df_exportar.index.name or 'index', df_exportar.index)


            try:
                df_exportar.to_excel(writer, sheet_name=nome_aba[:31], index=False)
                print(f"  [OK] Adicionado '{nome_original}' como aba '{nome_aba}'")
            except Exception as e:
                print(f"  [ERRO] Falha ao adicionar aba '{nome_aba}': {e}")

        # Aba de mapeamento
        try:
            df_mapa = pd.DataFrame(mapeamento_nomes)
            df_mapa.to_excel(writer, sheet_name='_mapa_de_nomes', index=False)
            print("  [INFO] Aba de mapeamento adicionada.")
        except Exception as e:
            print(f"  [ERRO] Falha ao adicionar aba de mapeamento: {e}")

        try:
            writer.close()
            print(f"[SUCESSO] Arquivo Excel salvo em: {caminho_arquivo_categoria}")
        except Exception as e:
            print(f"[ERRO] Falha ao salvar arquivo final: {e}")

    print(f"\n[FINALIZADO] Exportação concluída com arquivos Excel únicos por categoria.")


In [34]:
exportar_dados_organizados(todos_dados)

Iniciando a exportação para a pasta base: '../../dados_tratados/PNAD_2009'

--- Exportando categoria: AGRESSAO ---
  [OK] CSV salvo: ../../dados_tratados/PNAD_2009\csv\agressao\1b74a5d3a84c93f0545a532ab6390886.csv
  [OK] XLSX salvo: ../../dados_tratados/PNAD_2009\xlsx\agressao\1b74a5d3a84c93f0545a532ab6390886.xlsx
  [OK] CSV salvo: ../../dados_tratados/PNAD_2009\csv\agressao\2e1b5c8bbf547b0ddca6a272c88ae050.csv
  [OK] XLSX salvo: ../../dados_tratados/PNAD_2009\xlsx\agressao\2e1b5c8bbf547b0ddca6a272c88ae050.xlsx
  [OK] CSV salvo: ../../dados_tratados/PNAD_2009\csv\agressao\dc386ccdb2582580dbe9287b8e68023d.csv
  [OK] XLSX salvo: ../../dados_tratados/PNAD_2009\xlsx\agressao\dc386ccdb2582580dbe9287b8e68023d.xlsx
  [OK] CSV salvo: ../../dados_tratados/PNAD_2009\csv\agressao\db0267794865061ec415d73169ee1c5d.csv
  [OK] XLSX salvo: ../../dados_tratados/PNAD_2009\xlsx\agressao\db0267794865061ec415d73169ee1c5d.xlsx
  [OK] CSV salvo: ../../dados_tratados/PNAD_2009\csv\agressao\444035ee93236a197fd

In [35]:
exportar_dados_organizados_excel_unico(todos_dados)

Iniciando a exportação para a pasta base: '../../dados_tratados/PNAD_2009'

--- Exportando categoria: AGRESSAO ---
  [OK] Adicionado 'Coeficientes de variação das estimativas de pessoas de 10 anos ou mais de idade que foram vítimas de agressão física, no período de referência de 365 dias, por sexo e cor ou raça, segundo a situação do domicílio, as Grandes Regiões e as Unidades da Federação' como aba 'Coeficientes de variaç_1b74a5d3'
  [OK] Adicionado 'Coeficientes de variação das estimativas de distribuição das pessoas de 10 anos ou mais de idade que foram vítimas de agressão física, no período de referência de 365 dias, por sexo e cor ou raça, segundo a situação do domicílio, as Grandes Regiões e as Unidades da Federação' como aba 'Coeficientes de variaç_2e1b5c8b'
  [OK] Adicionado 'Coeficientes de variação das estimativas de percentual de pessoas que foram vítimas de agressão física, no período de referência de 365 dias, na população de 10 anos ou mais de idade, por sexo e cor ou raç

In [36]:
todos_dados['agressao']['Coeficiente de variação das estimativas de pessoas de 10 anos ou mais de idade que foram vítimas de agressão física, no período de referência de 365 dias, por sexo e cor ou raça, segundo algumas Unidades da Federação e Regiões Metropolitanas']

,total,homens,mulheres,branca,preta/parda
pará,7.58,7.90,11.18,15.12,7.49
região metropolitana de belém,8.57,10.07,11.52,16.38,9.57
ceará,8.67,8.19,13.53,14.51,8.98
região metropolitana de fortaleza,7.83,9.27,11.40,13.59,8.95
pernambuco,6.58,7.96,9.18,11.82,7.91
região metropolitana de recife,8.64,11.01,12.52,12.90,11.22
bahia,6.10,7.01,8.32,15.22,5.97
região metropolitana de salvador,6.23,7.76,9.18,13.71,6.63
minas gerais,7.32,8.49,8.57,11.86,7.64
região metropolitana de belo horizonte,9.10,11.38,12.17,17.64,10.36
